In [28]:
## Import Libraries

import pandas as pd
import numpy as np
import string
import re
import nltk
import imblearn

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.model_selection import GridSearchCV

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from imblearn.pipeline import Pipeline as imbPipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import FunctionTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import FeatureUnion


from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import PCA

from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import cohen_kappa_score
from sklearn.manifold import TSNE    

from imblearn.over_sampling import SMOTE

In [29]:
# load data
df_test = pd.read_csv('../data/processed/mtsamples_nlp.csv')
df_test.transcription=df_test.transcription.astype(str)

In [30]:
# retrieve labels as function
def get_labels(data):
    return data['medical_specialty'].tolist()

df_test_label = get_labels(df_test)

df_test_X = df_test['transcription_f'].astype(str)

In [31]:
# split data into train and test set (first split data into train and test set to only transform the train set)
def split_data(X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = split_data(df_test_X, df_test_label)

### Feature Exploration

In [96]:
# transform df_test_label to dataframe
df_label = pd.DataFrame(df_test_label)
df_label.columns = ['label']
df_label.head()

,label
0,Cardiovascular / Pulmonary
1,Cardiovascular / Pulmonary
2,Cardiovascular / Pulmonary
3,Cardiovascular / Pulmonary
4,Cardiovascular / Pulmonary


In [97]:
# vectorize df_test_X to dataframe
vectorizer = CountVectorizer()
df_test_X_vec = vectorizer.fit_transform(df_test_X)
df_test_X_vec = df_test_X_vec.toarray()
print(df_test_X_vec.shape)

df_test_X_vec = pd.DataFrame(df_test_X_vec, columns=vectorizer.get_feature_names_out ())
df_test_X_vec.head()

(2976, 2702)


,0007,005,01,0125,020,025,03,0395,05,050,...,zithromax,zocor,zofran,zoladex,zoloft,zometa,zone,zygoma,zygomatic,zyprexa
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [98]:
# join df_label and df_test_X_vec on index
df = df_test_X_vec.join(df_label)
# group by label and show which columns appear the most in each label
df.groupby('label').sum()

,0007,005,01,0125,020,025,03,0395,05,050,...,zithromax,zocor,zofran,zoladex,zoloft,zometa,zone,zygoma,zygomatic,zyprexa
label,,,,,,,,,,,,,,,,,,,,,
Cardiovascular / Pulmonary,0,0,1,0,0,0,0,0,1,0,...,1,1,0,0,0,0,0,0,0,0
Consult - History and Phy.,0,0,0,1,0,0,0,1,0,0,...,0,1,0,0,1,0,1,0,0,1
Gastroenterology,0,0,0,0,0,2,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
General Medicine,0,0,0,1,0,0,1,0,0,0,...,1,0,1,0,1,0,1,0,0,0
Neurology,1,1,0,0,0,0,0,1,1,0,...,1,1,0,0,0,0,0,0,0,1
Obstetrics / Gynecology,0,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,1,0,0,0
Orthopedic,1,0,0,0,0,5,0,0,16,1,...,1,0,0,0,0,0,0,0,0,0
Radiology,1,0,0,0,0,0,0,0,3,0,...,0,0,0,0,0,0,0,0,1,0
SOAP / Chart / Progress Notes,0,0,2,0,0,0,1,1,2,0,...,0,1,0,0,2,1,0,0,0,0


In [99]:
# show top symptoms with word count
def get_top_n_words(corpus, n=None):
    vec = CountVectorizer().fit(corpus)
    bag_of_words = vec.transform(corpus)
    sum_words = bag_of_words.sum(axis=0) 
    words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
    words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
    return words_freq[:n]

top_words = get_top_n_words(X_train, n=5)
print("Symptoms occuring the most in X_train:", top_words)

# show top symptoms per label with word count (on whole dataset, not yet split into train and test set and smoothed out)
top_words_label = df.groupby('label').sum().apply(lambda x: x.sort_values(ascending=False).head(10), axis=1)

top_Cardiovascular = top_words_label.iloc[0].loc[top_words_label.iloc[0]>0].sort_values(ascending=False)
print("Top Words in Cardiovascular/Pulmonary:\n", top_Cardiovascular)

top_Consult = top_words_label.iloc[1].loc[top_words_label.iloc[1]>0].sort_values(ascending=False)
print("Top Words in Consult - History and Phy.	:\n", top_Consult)

top_Gastroenterology = top_words_label.iloc[2].loc[top_words_label.iloc[2]>0].sort_values(ascending=False)
print("Top Words in Gastroenterology:\n", top_Gastroenterology)

top_General_Medicine = top_words_label.iloc[3].loc[top_words_label.iloc[3]>0].sort_values(ascending=False)
print("Top Words in General Medicine:\n", top_General_Medicine)

top_Neurology = top_words_label.iloc[4].loc[top_words_label.iloc[4]>0].sort_values(ascending=False)
print("Top Words in Neurology:\n", top_Neurology)

top_Obstetrics_Gynecology = top_words_label.iloc[5].loc[top_words_label.iloc[5]>0].sort_values(ascending=False)
print("Top Words in Obstetrics/Gynecology:\n", top_Obstetrics_Gynecology)

top_Orthopedic = top_words_label.iloc[6].loc[top_words_label.iloc[6]>0].sort_values(ascending=False)
print("Top Words in Orthopedic:\n", top_Orthopedic)

top_Radiology = top_words_label.iloc[7].loc[top_words_label.iloc[7]>0].sort_values(ascending=False)
print("Top Words in Radiology:\n", top_Radiology)

top_SOAP = top_words_label.iloc[8].loc[top_words_label.iloc[8]>0].sort_values(ascending=False)
print("Top Words in SOAP:\n", top_SOAP)

top_Surgery = top_words_label.iloc[9].loc[top_words_label.iloc[9]>0].sort_values(ascending=False)
print("Top Words in Surgery:\n", top_Surgery)

top_Urology = top_words_label.iloc[10].loc[top_words_label.iloc[10]>0].sort_values(ascending=False)
print("Top Words in Urology:\n", top_Urology)


Symptoms occuring the most in X_train: [('patient', 1945), ('left', 1072), ('blood', 890), ('skin', 837), ('tissue', 566)]
Top Words in Cardiovascular/Pulmonary:
 patient        218.0
left           169.0
artery         106.0
heart          106.0
blood          105.0
ventricular     81.0
wall            74.0
coronary        72.0
skin            66.0
lung            65.0
Name:  Cardiovascular / Pulmonary, dtype: float64
Top Words in Consult - History and Phy.	:
 patient    167.0
blood       89.0
left        71.0
abdomen     59.0
bowel       56.0
heart       56.0
lung        53.0
skin        47.0
eye         46.0
edema       44.0
Name:  Consult - History and Phy., dtype: float64
Top Words in Gastroenterology:
 patient      162.0
abdominal     83.0
abdomen       79.0
left          60.0
bowel         59.0
colon         59.0
blood         57.0
esophagus     49.0
skin          49.0
stomach       48.0
Name:  Gastroenterology, dtype: float64
Top Words in General Medicine:
 patient    97.0
bloo

### Feature Selection and Step by Step

In [25]:
# vectorize X_train 
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)

In [26]:
# smote oversampling
sm = SMOTE(random_state=42)
X_train_vec, y_train = sm.fit_resample(X_train_vec, y_train)
X_train_vec

<8921x2574 sparse matrix of type '<class 'numpy.int64'>'
	with 163353 stored elements in Compressed Sparse Row format>

In [18]:
# Remove highly correlated features
def decorrelate(X, threshold):
    X = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out ())
    corr_matrix = X.corr().abs()
    upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    X = X.drop(X[to_drop], axis=1)
    return X


decorrelate(X_train_vec, 0.85)

,0007,005,01,0125,020,025,03,0395,05,050,...,year,yellowish,yolk,zithromax,zocor,zofran,zoloft,zone,zygoma,zygomatic
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8916,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8917,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8918,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8919,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [11]:
class MyDecorrelator1(BaseEstimator, TransformerMixin):
    
    def __init__(self, threshold= 0.85):
        self.threshold = threshold

    def fit(self, X, y=None):  
        vectorizer = CountVectorizer()
        X = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names_out ())
        corr_matrix = X.corr().abs()
        upper = corr_matrix.where(np.triu(np.ones(corr_matrix.shape), k=1).astype(bool))
        to_drop = [column for column in upper.columns if any(upper[column] > self.threshold)]
        self.to_drop = to_drop
        return self

    def transform(self, X, y=None):
        X = X.drop(X[self.to_drop], axis=1)
        return X

In [19]:
class MyDecorrelator(BaseEstimator, TransformerMixin):
    
    def __init__(self, threshold= 0.85):
        self.threshold = threshold

    def fit(self, X, y=None):
        correlated_features = set()  
        X = pd.DataFrame(X)
        corr_matrix = X.corr()
        for i in range(len(corr_matrix.columns)):
            for j in range(i):
                if abs(corr_matrix.iloc[i, j]) > self.threshold: # we are interested in absolute coeff value
                    colname = corr_matrix.columns[i]  # getting the name of column
                    correlated_features.add(colname)
        self.correlated_features = correlated_features
        return self

    def transform(self, X, y=None):
        return (pd.DataFrame(X)).drop(labels=self.correlated_features, axis=1)

### Model Pipeline

In [32]:
# The “saga” solver is a variant of “sag” that also supports the non-smooth penalty="l1" 
# This is therefore the solver of choice for sparse multinomial logistic regression
# L1 tends to shrink coefficients to zero whereas L2 tends to shrink coefficients evenly. 
# L1 is therefore useful for feature selection, as we can drop any variables associated with coefficients that go to zero
model_pipeline = imbPipeline([
        ('preprocessing',CountVectorizer()),
        ('smote', SMOTE(random_state=42)),
        #('decorrelation', MyDecorrelator1()),
        ('classifier', LogisticRegression(random_state=42, multi_class='multinomial', solver='saga', penalty='l1')),
])

model_pipeline.fit(X_train, y_train)

/Users/hannahpetry/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


Pipeline(steps=[('preprocessing', CountVectorizer()),
                ('smote', SMOTE(random_state=42)),
                ('classifier',
                 LogisticRegression(multi_class='multinomial', penalty='l1',
                                    random_state=42, solver='saga'))])

In [33]:
# poor performance without fine tuning
lr = model_pipeline.fit(X_train, y_train)
y_pred = lr.predict(X_test)
category_list = df_test.medical_specialty.unique()
print(classification_report(y_test, y_pred, target_names=lr.classes_))

                                precision    recall  f1-score   support

    Cardiovascular / Pulmonary       0.38      0.40      0.39        52
    Consult - History and Phy.       0.25      0.23      0.24        40
              Gastroenterology       0.28      0.33      0.30        42
              General Medicine       0.06      0.08      0.07        25
                     Neurology       0.18      0.22      0.20        23
       Obstetrics / Gynecology       0.31      0.47      0.37        30
                    Orthopedic       0.23      0.30      0.26        57
                     Radiology       0.18      0.18      0.18        51
 SOAP / Chart / Progress Notes       0.19      0.20      0.20        35
                       Surgery       0.47      0.33      0.39       212
                       Urology       0.22      0.28      0.25        29

                      accuracy                           0.30       596
                     macro avg       0.25      0.27      0.26 

/Users/hannahpetry/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [34]:
# take best model from grid search and perform evaluation

param_grid = [
    { 'classifier__C': [0.01, 0.1, 1, 10],
      'classifier': [LogisticRegression(multi_class='multinomial', random_state=42, solver='saga', penalty='l1')],
      #'classifier__penalty': ['l1', 'l2', 'elasticnet'],
    }
]

def grid_search(X_train, y_train, model_pipeline, param_grid):
    search = GridSearchCV(model_pipeline, param_grid, cv=5)
    search.fit(X_train, y_train)
    print("Best parameter", search.best_params_)
    return search.best_estimator_
    
best_model = grid_search(X_train, y_train, model_pipeline, param_grid)
y_pred = best_model.predict(X_test)

category_list = df_test.medical_specialty.unique()
# predict and evaluate
print(classification_report(y_test, y_pred, target_names=best_model.classes_))

/Users/hannahpetry/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/hannahpetry/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/hannahpetry/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/hannahpetry/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
/Users/hannahpetry/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: Conver

Best parameter {'classifier': LogisticRegression(C=0.1, multi_class='multinomial', penalty='l1',
                   random_state=42, solver='saga'), 'classifier__C': 0.1}
                                precision    recall  f1-score   support

    Cardiovascular / Pulmonary       0.41      0.58      0.48        52
    Consult - History and Phy.       0.36      0.25      0.29        40
              Gastroenterology       0.50      0.55      0.52        42
              General Medicine       0.16      0.20      0.18        25
                     Neurology       0.29      0.35      0.31        23
       Obstetrics / Gynecology       0.37      0.67      0.48        30
                    Orthopedic       0.35      0.51      0.41        57
                     Radiology       0.17      0.14      0.15        51
 SOAP / Chart / Progress Notes       0.24      0.26      0.25        35
                       Surgery       0.62      0.41      0.49       212
                       Urology      

/Users/hannahpetry/opt/anaconda3/envs/nlp_masterthesis/lib/python3.10/site-packages/sklearn/linear_model/_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [35]:
print(best_model.named_steps['classifier'].coef_)
print(len(best_model.named_steps['classifier'].coef_[0]))

# how many coefficients are non zero?
print(len(best_model.named_steps['classifier'].coef_[0][best_model.named_steps['classifier'].coef_[0] != 0]))

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
2574
29


### Predictions

In [ ]:
# sample from test set
sample = X_test.iloc[13]
sample

"{'lip', 'abdomen', 'needle', 'endometrial', 'wall', 'anterior', 'cervix', 'tissue', 'fallopian', 'decidual', 'patient', 'tube', 'unclotted', 'liver', 'umbilicus', 'mesosalpinx', 'saline', 'vulsellum', 'vagina', 'left', 'midline', 'curettings', 'clot', 'fundus', 'peritoneum', 'omental', 'vesicouterine', 'uterus', 'uterine', 'blood'}"

In [138]:
# Prediction for sample from test set
sample = X_test.iloc[13]
print("Prediction:", best_model.predict([sample]))
# Actual category of first sample from test set
print("Actual:", y_test[13])
# Predict probabilities for a sample from test set
prob_array = best_model.predict_proba(X_test)[13,:]
prob_df = pd.DataFrame(prob_array, index=lr.classes_, columns=['Probability']).sort_values(by='Probability', ascending=False)
prob_df.Probability = prob_df.Probability.round(3)
print(prob_df)

Prediction: [' Obstetrics / Gynecology']
Actual:  Obstetrics / Gynecology
                                Probability
 Obstetrics / Gynecology              0.605
 Surgery                              0.283
 Cardiovascular / Pulmonary           0.032
 Gastroenterology                     0.020
 Radiology                            0.017
 Orthopedic                           0.011
 Neurology                            0.008
 Consult - History and Phy.           0.008
 SOAP / Chart / Progress Notes        0.006
 Urology                              0.005
 General Medicine                     0.004


In [139]:
to_pred = "lip abdomen needle endometrial wall anterior cervix"
def predict_probability(model: imblearn.pipeline.Pipeline, value) -> pd.DataFrame:
    """
    get probabilities for sample

    Parameters
    ----------
    model : imblearn.pipeline.Pipeline
        best model from train.py
    value : str
        sample
    category_list: list[str]
        list of unique labels

    Returns
    -------
    pd.DataFrame
        Probabilities for labels
    """

    prob_array = model.predict_proba(value)
    prob_df = pd.DataFrame(
        prob_array, index=["Probability"], columns=model.classes_
    ).transpose().sort_values(by="Probability", ascending=False)
    return prob_df

res_df = predict_probability(best_model, [to_pred])
res_df

,Probability
Obstetrics / Gynecology,0.299320
Radiology,0.148991
Gastroenterology,0.123282
Cardiovascular / Pulmonary,0.070136
Surgery,0.069127
General Medicine,0.062495
Consult - History and Phy.,0.061271
Neurology,0.050716
Urology,0.039040
SOAP / Chart / Progress Notes,0.038013
